# Create a Simple Solar System Model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import namedtuple

In [ ]:
class planet():
    "A planet in our solar system"
    def __init__(self,semimajor,eccetricity):
        self.x  =np.zeros(2)
        self.v  =np.zeros(2)
        self.a_g=np.zeros(2)
        self.t  =0.0
        self.dt =0.0
        self.a  =semimajor
        self.e  =eccentricity
        self.istep=0
        self.name =""

In [ ]:
solar_system = { "M_sun":1.0, "G":39.4784176043574320}

# Define some functions for setting circular velocity

In [ ]:
def SolarCircularVelocity(p):
    
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = (p.x[0]**2 + p.x[1]**2 )**0.5
    
    #return the circular velocity
    return (G*M/r)**0.5

# Write a function to compute the gravitational acceleration on each planet

In [ ]:
def SolarGravitationalAcceleration(p):
    
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = (p.x[0]**2 + p.x[1]**2 )**0.5
    
    #acceleration in AU/yr/yr
    a_grav = -1.0*G*M/r**2
    
    #find the angle at this position
    if(p.x[0]==0.0):
        if(p.x[1]>0.0):
            theta = 0.5*np.pi
        else:
            theta = 1.5*np.pi
    else:
        theta = np.arctan2(p.x[1],p.x[0])
    
    #set the x and y components of the velocity
    
    return a_grav*np.cos(theta), a_grav*np.sin(theta)

# Compute the timestep

In [ ]:
def calc_dt(p):
    
    #integration tolerance
    ETA_TIME_STEP = 0.0004
    
    #compute timestep
    eta = ETA_TIME_STEP
    v=(p.v[0]**2 + p.v[1]**2)**0.5
    a=(p.a_g[0]**2 + p.a_g[1]**2)**0.5
    dt=eta*np.fmin(1./np.fabs(v),1./np.fabs(a)**0.5)
    
    return dt

# Define the initial conditions

In [ ]:
def SetPlanet(p,i):
    
    AU_in_km = 1.495979e+8  #AU in km
    
    #circular velocity
    v_c = 0.0       #circular v in AU/yr
    v_e = 0.0       #v at perihelion in AU/yr
    
    #Mercury
    if(i==0):
        
        p.a = 57909227.0/AU_in_km
        
        p.e = 0.20563593
        
        p.name = "Mercury"
        
    elif(i==1):
        
        p.a = 108209475.0/AU_in_km
        
        p.e = 0.00677672
        
        p.name = "Venus"
        
    elif(i==2):
        
        p.a = 1.0/AU_in_km
        
        p.e = 0.01671123
        
        p.name = "Earth"
        
    #set remaining properties
    p.t = 0.0
    p.x[0] = p.a*(1.0-p.e)
    p.x[1] = 0.0
    
    #get equivalent circular velocity
    v_c=SolarCircularVelocity(p)
    v_e=v_c*(1+p.e)**0.5
    
    #set velocity
    p.v[0] = 0.0
    p.v[1] = v_e
    
    #calculate gravitational acceleration from Sun
    p.a_g = SolarGravitationalAcceleration(p)
    
    p.dt = calc_dt(p)

# Write leapfrog integrator

In [ ]:
def x_first_step(x_i, v_i, a_i, dt):
    
    return x_i + 0.5*v_i*dt + 0.25*a_i*dt**2

In [ ]:
def v_full_step(v_i, a_ipoh, dt):
    
    return v_i + a_ipoh*dt;

In [ ]:
def x_full_step(x_ipoh, v_ipl, a_ipoh, dt):
    
    return x_ipoh + v_ipl*dt